# Notable differences between Polars and Pandas
* Polars is significantly faster than Pandas if you follow their api and strategies
* Polars has lazy execution, which allows the backend to decide which operations to run in order to minimize overhead, through parallization
* Polars is based on `rust`, so it has a smaller memory footprint
* Polars doesn't use indexes, and has strict datatypes; for example Pandas allows dictionaries in a Series

# Installing Polars
* In your environment, install polwars with `pip install polars`
* If you want all dependencies, you can install with `pip install polars[all]`.
* For me...

```bash
pip install polars[pandas,numpy,pyarrow]
```

# Basic Polar manipulations
* this data is a processed form of SemMedDB, a dataset of triples (subject-predicate-object) facts backed by a publication (pmid) and the date it was published (pub_years)
* start_id, and end_id are unique entity identifiers, we can look them up in a separate file

## reading dataframes in, creating a test dataset

In [1]:
import polars as pl # polars
import pandas as pd # pandas

In [2]:
df = pl.read_csv(source="../data/time_networks-6_metanode/2023/edges.csv")
df.head()

:START_ID,:END_ID,:TYPE,pmids,n_pmids:INT,pub_years,first_pub:INT
str,str,str,str,i64,str,i64
"""C1417544""","""D016676""","""ADMINISTERED_T…","""['35694306']""",1,"""[2022]""",2022
"""D016334""","""D006367""","""ADMINISTERED_T…","""['12431786']""",1,"""[2002]""",2002
"""C3147220""","""C0597694""","""ADMINISTERED_T…","""['2060650']""",1,"""[1991]""",1991
"""C0919506""","""C1513528""","""ADMINISTERED_T…","""['8625325']""",1,"""[1996]""",1996
"""C3147220""","""C0882849""","""ADMINISTERED_T…","""['16109763']""",1,"""[2005]""",2005


Reading, Renaming the headers, exporting

In [3]:
%%time
pl.read_csv(source="../data/time_networks-6_metanode/2023/edges.csv").sample(
    fraction=0.05
).rename(
    {
        ":START_ID": "start_id",
        ":END_ID": "end_id",
        ":TYPE": "type",
        "n_pmids:INT": "n_pmids",
        "first_pub:INT": "first_pub",
    }
).write_csv(
    "../data/time_networks-6_metanode/2023/edges_five_percent.csv"
)

CPU times: user 10.5 s, sys: 3.7 s, total: 14.2 s
Wall time: 3.86 s


## Selecting columns

In [4]:
df.select([":START_ID", ":END_ID"]).head()

:START_ID,:END_ID
str,str
"""C1417544""","""D016676"""
"""D016334""","""D006367"""
"""C3147220""","""C0597694"""
"""C0919506""","""C1513528"""
"""C3147220""","""C0882849"""


In [5]:
df[[":START_ID", ":END_ID"]].head()

:START_ID,:END_ID
str,str
"""C1417544""","""D016676"""
"""D016334""","""D006367"""
"""C3147220""","""C0597694"""
"""C0919506""","""C1513528"""
"""C3147220""","""C0882849"""


## Manipulating column data
* unlike pandas, polars does not support direct column assignment
* the `with_columns` function runs all column manipulations in parallel

### Check if a `n_pmids` is greater than 1

In pandas, you can directly assign your columns to your dataframe...

In [6]:
df_pandas = df.to_pandas()

In [7]:
%%timeit
df_pandas["n_pmid_greater1"] = df_pandas["n_pmids:INT"].apply(lambda x: x > 1)

1.15 s ± 18.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
df_pandas.head(2)

,:START_ID,:END_ID,:TYPE,pmids,n_pmids:INT,pub_years,first_pub:INT,n_pmid_greater1
0,C1417544,D016676,ADMINISTERED_TO_GatA,['35694306'],1,[2022],2022,False
1,D016334,D006367,ADMINISTERED_TO_GatA,['12431786'],1,[2002],2002,False


* In polars you have to do it in the operation and assign a column, otherwise its inplace.
* Note that polars doesn't have indexes
* Note that some operations, while operations can be written the same way as in pandas, its better to do things the polars way; See below with their helpful hints.

In [9]:
%%timeit
df.with_columns(
    pl.col("n_pmids:INT").map_elements(lambda x: x > 1).alias("n_pmid_greater1"),
).with_columns(pl.col('n_pmid_greater1').count()))

<magic-timeit>:2: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower than the native expressions API.
Only use if you absolutely CANNOT implement your logic otherwise.
In this case, you can replace your `map_elements` with the following:
  - pl.col("n_pmids:INT").map_elements(lambda x: ...)
  + pl.col("n_pmids:INT") > 1

<magic-timeit>:2: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower than the native expressions API.
Only use if you absolutely CANNOT implement your logic otherwise.
In this case, you can replace your `map_elements` with the following:
  - pl.col("n_pmids:INT").map_elements(lambda x: ...)
  + pl.col("n_pmids:INT") > 1

<magic-timeit>:2: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower than the native expressions API.
Only use if you absolutely CANNOT implement your logic otherwise.
In this case, you can replace your `map_elements` with the following:
  - pl.col("n_pmids:INT").map_elements(lambda x: ...)

1.27 s ± 16.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
# you can also do
df.with_columns((pl.col("n_pmids:INT") > 1).alias("n_pmid_greater1"))

6.03 ms ± 31.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
df = df.with_columns((pl.col("n_pmids:INT") > 1).alias("n_pmid_greater1"))
df.head(2)

:START_ID,:END_ID,:TYPE,pmids,n_pmids:INT,pub_years,first_pub:INT,n_pmid_greater1
str,str,str,str,i64,str,i64,bool
"""C1417544""","""D016676""","""ADMINISTERED_T…","""['35694306']""",1,"""[2022]""",2022,false
"""D016334""","""D006367""","""ADMINISTERED_T…","""['12431786']""",1,"""[2002]""",2002,false


## Getting a unique set of identifiers for each publication year
* lets say we want to get the unique set of start_ids for each first_pub year, and get how long it is

In pandas we would write..

In [12]:
%%time
df_pd_groupby = (
    df_pandas.groupby("first_pub:INT")
    .agg(list)
    .reset_index()[["first_pub:INT", ":START_ID"]]
)
df_pd_groupby[":START_ID"] = df_pd_groupby[":START_ID"].apply(lambda x: list(set(x)))
df_pd_groupby["count"] = df_pd_groupby[":START_ID"].apply(lambda x: len(x))
df_pd_groupby = df_pd_groupby.sort_values(by="count", ascending=False)
df_pd_groupby.head(2)

CPU times: user 9.36 s, sys: 1.21 s, total: 10.6 s
Wall time: 10.6 s


,first_pub:INT,:START_ID,count
240,2021,"[D013127, C0856053, C1156019, C0459847, D00357...",57653
241,2022,"[D013127, C1156019, C0459847, D003571, C141616...",57432


In polars we would write..

In [13]:
%%time
df_groupby = (
    df.group_by("first_pub:INT")
    .agg(":START_ID")
    .with_columns(pl.col(":START_ID").list.unique())
    .with_columns(pl.col(":START_ID").list.len().alias("count"))
    .sort(by="count", descending=True)
)
df_groupby.head(2)

CPU times: user 3.46 s, sys: 1min 33s, total: 1min 37s
Wall time: 2.24 s


first_pub:INT,:START_ID,count
i64,list[str],u32
2021,"[""C037720"", ""C476933"", … ""C1419228""]",57653
2022,"[""D051378"", ""C099718"", … ""C1333258""]",57432


## merging in names for each particular triple
* here we're looking at joins

In [14]:
%%time
nodes_pd = pd.read_csv('../data/time_networks-6_metanode/2023/nodes.csv')

CPU times: user 144 ms, sys: 11.6 ms, total: 156 ms
Wall time: 155 ms


In [15]:
%%time
nodes_pl = pl.read_csv('../data/time_networks-6_metanode/2023/nodes.csv')

CPU times: user 102 ms, sys: 175 ms, total: 277 ms
Wall time: 14 ms


In [16]:
nodes_pl.head(2)

cui:ID,name,:LABEL,id_source
str,str,str,str
"""C1518046""","""Lymph Node Eff…","""Anatomy""","""UMLS"""
"""C1278805""","""Entire nasopha…","""Anatomy""","""UMLS"""


In [17]:
%%time
pd.merge(
    left=df_pandas,
    right=nodes_pd[["cui:ID", "name", ":LABEL"]],
    left_on=":START_ID",
    right_on="cui:ID",
    how="left",
).rename(
    columns={":START_ID": "start_id", "name": "start_name", ":LABEL": "start_label"}
)

CPU times: user 3.31 s, sys: 1.56 s, total: 4.87 s
Wall time: 4.86 s


,start_id,:END_ID,:TYPE,pmids,n_pmids:INT,pub_years,first_pub:INT,n_pmid_greater1,cui:ID,start_name,start_label
0,C1417544,D016676,ADMINISTERED_TO_GatA,['35694306'],1,[2022],2022,False,C1417544,MYH9 gene,Genes & Molecular Sequences
1,D016334,D006367,ADMINISTERED_TO_GatA,['12431786'],1,[2002],2002,False,D016334,"Genes, vpr",Genes & Molecular Sequences
2,C3147220,C0597694,ADMINISTERED_TO_GatA,['2060650'],1,[1991],1991,False,C3147220,ERVK-25,Genes & Molecular Sequences
3,C0919506,C1513528,ADMINISTERED_TO_GatA,['8625325'],1,[1996],1996,False,C0919506,FGF2 gene,Genes & Molecular Sequences
4,C3147220,C0882849,ADMINISTERED_TO_GatA,['16109763'],1,[2005],2005,False,C3147220,ERVK-25,Genes & Molecular Sequences
...,...,...,...,...,...,...,...,...,...,...,...
9784636,C1539082,D057134,higher_than_GdfCD,"['6261091', '9203727']",2,"[1980, 1997]",1980,True,C1539082,CD40LG gene,Genes & Molecular Sequences
9784637,C1366646,D007987,higher_than_GdfCD,"['6307754', '2859018', '2875442', '3007215']",4,"[1983, 1985, 1986, 1986]",1983,True,C1366646,VIP gene,Genes & Molecular Sequences
9784638,C1366628,D018048,higher_than_GdfCD,"['2996968', '6297936']",2,"[1985, 1982]",1982,True,C1366628,ATP8A2 gene,Genes & Molecular Sequences
9784639,C1539630,C084541,higher_than_GdfCD,"['24680483', '24865692', '21377199']",3,"[2014, 2014, 2011]",2011,True,C1539630,HBEGF gene,Genes & Molecular Sequences


In [18]:
%%time
df.join(
    other=nodes_pl[["cui:ID", "name", ":LABEL"]],
    left_on=":START_ID",
    right_on="cui:ID",
    how="left",
).rename({":START_ID": "start_id", "name": "start_name", ":LABEL": "start_label"})

CPU times: user 4.09 s, sys: 7.48 s, total: 11.6 s
Wall time: 637 ms


start_id,:END_ID,:TYPE,pmids,n_pmids:INT,pub_years,first_pub:INT,n_pmid_greater1,start_name,start_label
str,str,str,str,i64,str,i64,bool,str,str
"""C1417544""","""D016676""","""ADMINISTERED_T…","""['35694306']""",1,"""[2022]""",2022,false,"""MYH9 gene""","""Genes & Molecu…"
"""D016334""","""D006367""","""ADMINISTERED_T…","""['12431786']""",1,"""[2002]""",2002,false,"""Genes, vpr""","""Genes & Molecu…"
"""C3147220""","""C0597694""","""ADMINISTERED_T…","""['2060650']""",1,"""[1991]""",1991,false,"""ERVK-25""","""Genes & Molecu…"
"""C0919506""","""C1513528""","""ADMINISTERED_T…","""['8625325']""",1,"""[1996]""",1996,false,"""FGF2 gene""","""Genes & Molecu…"
"""C3147220""","""C0882849""","""ADMINISTERED_T…","""['16109763']""",1,"""[2005]""",2005,false,"""ERVK-25""","""Genes & Molecu…"
"""C1417532""","""D032446""","""ADMINISTERED_T…","""['20069545']""",1,"""[2010]""",2010,false,"""MYF5 gene""","""Genes & Molecu…"
"""C1426700""","""D009865""","""ADMINISTERED_T…","""['15271716']""",1,"""[2004]""",2004,false,"""SLC25A29 gene""","""Genes & Molecu…"
"""C3147220""","""D001792""","""ADMINISTERED_T…","""['7041120']""",1,"""[1982]""",1982,false,"""ERVK-25""","""Genes & Molecu…"
"""C1384512""","""D005347""","""ADMINISTERED_T…","""['12548290']""",1,"""[2003]""",2003,false,"""CSN2 gene""","""Genes & Molecu…"


# Speed Test evaluation of Polars
* create a test dataset * about 5% of the original dataset, ~50mb
* Test awk, polars and pandas

## Test Awk
* to be fair, it was slightly faster in terminal using `time`
```bash
real    0m0.939s
user    0m0.931s
sys     0m0.009s
```

In [19]:
%%timeit
!awk -F',' '($NF > 1970){year[$NF]++}; END{for(i in year){print i, year[i]}}' ../data/time_networks-6_metanode/2023/edges_five_percent.csv | sort -k2,2rn >> ../data/time_networks-6_metanode/2023/edges_five_percent_result_awk.csv

1.34 s ± 6.84 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Test Pandas

In [20]:
%%timeit
import pandas as pd

(
    pd.read_csv("../data/time_networks-6_metanode/2023/edges_five_percent.csv")
    .query("first_pub>1970")
    .groupby("first_pub")
    .count()
    .reset_index()[["first_pub", "start_id"]]
    .sort_values(by="start_id", ascending=False)
    .to_csv("../data/time_networks-6_metanode/2023/edges_five_percent_pnds_res.csv")
)

803 ms ± 8.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Test Polars

In [21]:
%%timeit
import polars as pl

pl.read_csv("../data/time_networks-6_metanode/2023/edges_five_percent.csv").filter(
    pl.col("first_pub") > 1970
).group_by("first_pub").count().sort(by="count", descending=True).write_csv(
    "../data/time_networks-6_metanode/2023/edges_five_percent_result.csv"
)

466 ms ± 53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Test on a bigger 9 million lined dataset

In [22]:
print(
    f'Dataframe size: {pl.read_csv("../data/time_networks-6_metanode/2023/edges.csv").shape[0]:,}'
)

Dataframe size: 9,784,641


In [23]:
%%timeit
!awk -F',' '($NF > 1970){year[$NF]++}; END{for(i in year){print i, year[i]}}' ../data/time_networks-6_metanode/2023/edges.csv | sort -k2,2rn >> ../data/time_networks-6_metanode/2023/edges_result_awk.csv

18.9 s ± 96.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%%timeit
import pandas as pd

(
    pd.read_csv("../data/time_networks-6_metanode/2023/edges.csv")
    .rename(columns={":START_ID": "start_id", "first_pub:INT": "first_pub"})
    .query("first_pub>1970")
    .groupby("first_pub")
    .count()
    .reset_index()[["first_pub", "start_id"]]
    .sort_values(by="start_id", ascending=False)
    .to_csv("../data/time_networks-6_metanode/2023/edges_pnds_res.csv")
)

15.9 s ± 319 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%%timeit
import polars as pl

pl.read_csv("../data/time_networks-6_metanode/2023/edges.csv").filter(
    pl.col("first_pub:INT") > 1970
).group_by("first_pub:INT").count().sort(by="count", descending=True).write_csv(
    "../data/time_networks-6_metanode/2023/edges_plrs_res.csv"
)

4.08 s ± 118 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
